# **Proyek Predictive Analytics: [Crop Recommendation Dataset]**

- **Nama** : Satria Dirgantara Nuryaman
- **Email** : satriadirgantaranuryaman15@gmail.com
- **ID Dicoding** : Satria Dirgantara Nuryaman 

## **Import Library**

In [2]:
# Data Manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

# Data Processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

# Model Machine Learning
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Model Evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Model Tuning
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Warnings
import warnings 
warnings.filterwarnings('ignore')

## **Data Loading**

In [3]:
FILE_PATH = 'data/Crop_recommendation.csv'
df = pd.read_csv(FILE_PATH)
display(df.head(), df.tail())

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


,N,P,K,temperature,humidity,ph,rainfall,label
2195,107,34,32,26.774637,66.413269,6.780064,177.774507,coffee
2196,99,15,27,27.417112,56.636362,6.086922,127.924610,coffee
2197,118,33,30,24.131797,67.225123,6.362608,173.322839,coffee
2198,117,32,34,26.272418,52.127394,6.758793,127.175293,coffee
2199,104,18,30,23.603016,60.396475,6.779833,140.937041,coffee


Dataset Crop Recommendation ini memiliki:
- 2200 baris.
- 8 fitur yang bernama N, P, K, temperature, humidity, ph, rainfall, dan label. 

## **Exploratory Data Analysis (EDA)**

### **Deskripsi Variable Dataset**


| Fitur        | Deskripsi                                      |
|--------------|------------------------------------------------|
| N            | Rasio kandungan Nitrogen dalam tanah           |
| P            | Rasio kandungan Phosphorous dalam tanah        |
| K            | Rasio kandungan Potassium dalam tanah          |
| temperature  | Suhu lingkungan (°C)                           |
| humidity     | Kelembapan relatif (%)                         |
| ph           | Nilai pH tanah                                 |
| rainfall     | Curah hujan (mm)                               |
| label        | Jenis tanaman yang direkomendasikan (target)   |

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB


Seperti yang sudah di jelaskan dataset ini memiliki 2200 baris yang terdiri dari 8 Fitur yang berbeda beda tipe datanya:

- temperature, humidity, ph, dan rainfall bertipe data float64.
- N (Nitrogen), P (Phosphorous), dan K (Potassium) bertipe data float64.
- label bertipe data object.

Bisa kita lihat juga ternyata tidak ada missing values di dataset kita ini, inkonsistensi data juga tidak ada, semuanya sudah clean, tapi kita harus memastikan nya lagi.

In [5]:
df.describe()

,N,P,K,temperature,humidity,ph,rainfall
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,25.616244,71.481779,6.469480,103.463655
std,36.917334,32.985883,50.647931,5.063749,22.263812,0.773938,54.958389
min,0.000000,5.000000,5.000000,8.825675,14.258040,3.504752,20.211267
25%,21.000000,28.000000,20.000000,22.769375,60.261953,5.971693,64.551686
50%,37.000000,51.000000,32.000000,25.598693,80.473146,6.425045,94.867624
75%,84.250000,68.000000,49.000000,28.561654,89.948771,6.923643,124.267508
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117


Berdasarkan analisis deskriptif terhadap 2.200 sampel, ditemukan bahwa fitur fitur lingkungan dan tanah memiliki sebaran nilai yang cukup luas. 

- Unsur hara tanah seperti nitrogen (N: 0 – 140 ppm), fosfor (P: 5 – 145 ppm), dan kalium (K: 5 – 205 ppm) menunjukkan variasi yang signifikan, menandakan perbedaan kondisi tanah yang cukup ekstrem antar wilayah. 
- Kandungan kalium secara khusus dan ajaib memiliki standar deviasi tertinggi (50.6), menunjukkan keragaman yang besar dan kemungkinan peran pentingnya dalam penentuan jenis tanaman yang sesuai.
- Rata rata suhu sebesar 25.6 C dengan rentang 8.8 – 43.7 C, serta kelembapan 71.5% (14.3 – 99.9%), mengindikasikan bahwa dataset ini mencakup berbagai kondisi iklim. pH tanah umumnya netral hingga sedikit asam/basa (rata rata 6.47), dan curah hujan berkisar antara 20.2 – 298.6 mm, mendukung analisis untuk berbagai jenis tanaman yang tumbuh di lingkungan berbeda.

Dengan kondisi lingkungan yang seberagam ini, penting untuk tahu fitur mana saja yang benar benar berpengaruh terhadap pilihan tanaman. Ini bukan cuma penting untuk akurasi model, tapi juga bisa jadi pegangan nyata bagi petani dalam mengelola lahannya dengan lebih tepat.

### **Identifikasi Missing Values**

In [8]:
missing_values = df.isnull().sum()
missing_values_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing Values' : missing_values,
    'Percentage' : missing_values_percentage
}) 

display(missing_df[missing_df['Missing Values'] > 0])

,Missing Values,Percentage


Ternyata benar informasi kita di atas, tidak ada nya missing values disini.

### **Mengecek Duplicated Data**

In [9]:
print(f'Jumlah data yang terduplikasi adalah {df.duplicated().sum()}')

Jumlah data yang terduplikasi adalah 0


Ternyata dataset Crop Recommendation sudah bersih, di buktikan dengan ketidakadaan nilai yang NaN dan juga duplikasi.